<a href="https://colab.research.google.com/github/mohdkhairilabdulkarim/STQD6014-Data-Science/blob/main/Week12_DataAggregation_20240107_WC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 12: Data Aggregation and Group Operation**
Please refer to the textbook: **"Python for Data Analysis"** by Wes McKinney for details of this topic on **Chapter 10**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Start by importing NumPy and pandas**

In [2]:
import numpy as np
import pandas as pd

# **10.1 Group Operations**
## The popular ***split-apply-combine***

In [3]:
# Example DataFrame 1
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : rng.standard_normal(7),
                   "data2" : rng.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [6]:
# Extract data1 and call groupby with the key1 column
# Create an intermediate file -> grouped
grouped = df['data1'].groupby(df['key1'])
grouped.head()

0   -1.423825
1    1.263728
3   -0.259173
4   -0.075343
5   -0.740885
Name: data1, dtype: float64

In [8]:
# Compute median for the intermediate grouped variable
# median() -> aggregation function -> 'transform' the data into single value
grouped.median()

key1
a   -0.740885
b   -0.167258
Name: data1, dtype: float64

In [9]:
# Another way
# Compute median without intermediate file
df['data1'].groupby(df['key1']).median()

key1
a   -0.740885
b   -0.167258
Name: data1, dtype: float64

In [11]:
# Another example: passing multiple groupby arrays of keys to find mean
# Passing multiple groupby arguments as a list
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     1      -1.423825
      2       1.263728
b     1      -0.075343
      2      -0.259173
Name: data1, dtype: float64

In [12]:
# Unstack
# transform the table -> from long (vertical) to wide (horizontal) format
means.unstack()

key2,1,2
key1,,
a,-1.423825,1.263728
b,-0.075343,-0.259173


In [13]:
# Display df content
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [14]:
# Pass column name as the groupby key
# Find mean for the whole data frame
df.groupby(df['key1']).mean()

,key2,data1,data2
key1,,,
a,1.5,-0.300327,0.083521
b,1.5,-0.167258,1.657953


In [16]:
# Next example: "key1" is not numeric
# The resulting DataFrame will only include the mean values data1 and data2 that are numeric
# "key1" doesn't appear in the output.
df.groupby(df['key2']).mean(numeric_only=True)

,data1,data2
key2,,
1,-0.934406,0.141681
2,0.502278,1.354234


### **Iterating over Groups**
***groupby*** supports ***iteration***, generating ***a sequence of 2-tuples***

In [17]:
# Display df content
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [19]:
# Iteration using a for loop
# To access individual data for each key1 value -> 'a' and 'b'
# name -> individual key1 values
# group -> data associated with the 'name'
for name, group in df.groupby(df['key1']):
    print(name)
    print(group)
    print("\n----------\n")

a
  key1  key2     data1     data2
0    a     1 -1.423825  0.648893
1    a     2  1.263728  0.361058
5    a  <NA> -0.740885 -0.759387

----------

b
  key1  key2     data1     data2
3    b     2 -0.259173  2.347410
4    b     1 -0.075343  0.968497

----------



In [20]:
# Display df content
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


# **Using tuples**

In [21]:
# In the case of multiple keys
# first element will be a tuple of key values
for (k1, k2), group in df.groupby([df['key1'],df['key2']]):
    print((k1,k2))
    print(group)
    print("\n----------\n")

('a', 1)
  key1  key2     data1     data2
0    a     1 -1.423825  0.648893

----------

('a', 2)
  key1  key2     data1     data2
1    a     2  1.263728  0.361058

----------

('b', 1)
  key1  key2     data1     data2
4    b     1 -0.075343  0.968497

----------

('b', 2)
  key1  key2     data1    data2
3    b     2 -0.259173  2.34741

----------



# **Using dictionary method -> {key:value} pair**

In [22]:
# Output as a dictionary
# Compute a dictionary using one-liner
pieces = {name:group for name, group in df.groupby(df['key1'])}
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -1.423825  0.648893
 1    a     2  1.263728  0.361058
 5    a  <NA> -0.740885 -0.759387,
 'b':   key1  key2     data1     data2
 3    b     2 -0.259173  2.347410
 4    b     1 -0.075343  0.968497}

In [24]:
# Accessing individual item in dictionary
# Using the "key"
pieces['a']

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
5,a,<NA>,-0.740885,-0.759387


In [25]:
# Accessing individual item in dictionary
# Using the "key"
pieces['b']

,key1,key2,data1,data2
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497


### **Grouping with Dictionaries and Series**

In [33]:
# Example DataFrame
# setting the seed number to ensure reproducibility
rng = np.random.default_rng(seed=12345)
people = pd.DataFrame(rng.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [34]:
# Add a few NA values
# Using the explicit indexer -> loc method
people.loc['Wanda',['b','c']] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,NaN,NaN,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [35]:
# Add a few more NA values
# Using the implicit indexer -> iloc method
# (4, 1) -> rows
# (0, 3) -> columns
people.iloc[(4,1),(0,3)] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863
Wanda,2.347410,NaN,NaN,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189


In [36]:
# Group correspondence for the same columns
# Mapping using a dictionary
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange", "g" : "violet"}

In [37]:
# Mapping and compute sum
# Using the people dataset
# Example from the people dataset: "-0.870662 - 0.259173" = -1.129835 (blue)
people.groupby(mapping, axis=1).sum()

,blue,red
Joe,-1.129835,-0.235440
Steve,0.648893,-4.061540
Wanda,0.902198,1.880456
Jill,-0.680811,2.127134
Trey,-0.718663,-0.457888


In [38]:
# Challenge: add the column f and g
# Using the assign() function
rng = np.random.default_rng(seed=12345)
people = people.assign(f = rng.standard_normal((5,1)),
                       g = rng.standard_normal((5,1)))
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863,1.263728,-1.367793
Wanda,2.347410,NaN,NaN,0.902198,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [39]:
# Mapping and compute sum again
people.groupby(mapping, axis=1).sum()

,blue,orange,red,violet
Joe,-1.129835,-1.423825,-0.235440,-0.740885
Steve,0.648893,1.263728,-4.061540,-1.367793
Wanda,0.902198,-0.870662,1.880456,0.648893
Jill,-0.680811,-0.259173,2.127134,0.361058
Trey,-0.718663,-0.075343,-0.457888,-1.952863


# **10.2 Data Aggregation**
* we want to transform the data into one single value
* sum, median, etc

In [40]:
# Display df content from previous example
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [44]:
# Create an intermediate grouped variable
grouped = df.groupby('key1')

In [46]:
# Select smallest values
grouped['data1'].nsmallest(1)

key1   
a     0   -1.423825
b     3   -0.259173
Name: data1, dtype: float64

In [45]:
# Select smallest values
grouped['data1'].nsmallest(2)

key1   
a     0   -1.423825
      5   -0.740885
b     3   -0.259173
      4   -0.075343
Name: data1, dtype: float64

In [47]:
# Define a peak_to_peak functions
# getting the range -> max - min
# return -> return the computed value
def peak_to_peak(arr):
    return arr.max() - arr.min()

# ***The agg() function in pandas***

* helps us <font color='red'>***break down our data into groups***</font>
* <font color='red'>***performs calculations or apply functions to each group***</font>
* <font color='red'>***returned aggregated results -> one single value***</font>
* a powerful tool for data analysis and summarization

In [48]:
# apply the peak_to_peak function to the dataframe
# to get range of data
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.687553,1.408280
b,1,0.183830,1.378913


In [49]:
# Manual calculation to double check
# range of data2 for key1 = a
0.648893 - (-0.759387)

1.40828

In [50]:
# The describe() -> generate a descriptive statistical summary of each group in a grouped DataFrame
# giving a quick overview of the data distribution within each group
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.300327  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.167258  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.261422  1.263728   3.0  0.083521  0.744032 -0.759387 -0.199165   
b    -0.121301 -0.075343   2.0  1.657953  0.975039  0.968497  1.313225   

                                    
           50%       75%       max  
key1                                
a     0.361058  0.504975  0.648893  
b     1.657953  2.002681  2.347410  

[2 rows x 24 columns]

In [ ]:
# To know more information about the describe() function
grouped.describe?

## **Column-Wise and Multiple Function Application**

In [51]:
# Get the tipping dataset from github
# Can download here: https://bit.ly/3VyE0vP
tips = pd.read_csv('https://bit.ly/3VyE0vP')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [52]:
# Add additional column: tip_pct
# assign()
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [53]:
# Group the tips by day and smoker
# Create an intermediate grouped object
grouped = tips.groupby(['day','smoker'])

In [54]:
# Extract the tip_pct from grouped object
# groupby in groupby
# Compute mean on 'tip_pct'
# Using the agg() function
# 'mean' -> is not a string -> build-in function in python
grouped_pct = grouped['tip_pct']
grouped_pct.agg(['mean'])

mean
day  smoker          
Fri  No      0.151650
     Yes     0.174783
Sat  No      0.158048
     Yes     0.147906
Sun  No      0.160113
     Yes     0.187250
Thur No      0.160298
     Yes     0.163863

In [55]:
# Passing multiple aggregation functions as a list
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [56]:
# Give customised name to the list of aggregated function
grouped_pct.agg([('Average',np.mean),('Stand_Deviation',np.std)])

Average  Stand_Deviation
day  smoker                           
Fri  No      0.151650         0.028123
     Yes     0.174783         0.051293
Sat  No      0.158048         0.039767
     Yes     0.147906         0.061375
Sun  No      0.160113         0.042347
     Yes     0.187250         0.154134
Thur No      0.160298         0.038774
     Yes     0.163863         0.039389

In [57]:
# Specify a list of functions to apply
# 'tip_pct' and 'total_bill'
funcs = ['count','mean','max','min','median']
res = grouped[['tip_pct','total_bill']].agg(funcs)
res

tip_pct                                         total_bill  \
              count      mean       max       min    median      count   
day  smoker                                                              
Fri  No           4  0.151650  0.187735  0.120385  0.149241          4   
     Yes         15  0.174783  0.263480  0.103555  0.173913         15   
Sat  No          45  0.158048  0.291990  0.056797  0.150152         45   
     Yes         42  0.147906  0.325733  0.035638  0.153624         42   
Sun  No          57  0.160113  0.252672  0.059447  0.161665         57   
     Yes         19  0.187250  0.710345  0.065660  0.138122         19   
Thur No          45  0.160298  0.266312  0.072961  0.153492         45   
     Yes         17  0.163863  0.241255  0.090014  0.153846         17   

                                              
                  mean    max    min  median  
day  smoker                                   
Fri  No      18.420000  22.75  12.46  19.235  
     Yes     16.813333  40.17   5.75  13.420  
Sat  No      19.661778  48.33   7.25  17.820  
     Yes     21.276667  50.81   3.07  20.390  
Sun  No      20.506667  48.17   8.77  18.430  
     Yes     24.120000  45.35   7.25  23.100  
Thur No      17.113111  41.19   7.51  15.950  
     Yes     19.190588  43.11  10.34  16.470

In [58]:
# Accessing individual data from aggregation result
# 'total_bill'
res['total_bill']

count       mean    max    min  median
day  smoker                                        
Fri  No          4  18.420000  22.75  12.46  19.235
     Yes        15  16.813333  40.17   5.75  13.420
Sat  No         45  19.661778  48.33   7.25  17.820
     Yes        42  21.276667  50.81   3.07  20.390
Sun  No         57  20.506667  48.17   8.77  18.430
     Yes        19  24.120000  45.35   7.25  23.100
Thur No         45  17.113111  41.19   7.51  15.950
     Yes        17  19.190588  43.11  10.34  16.470

In [62]:
# Another way: tuples
# Can pass aggreation functions as a list of tuples
tuples = [('AvErAGe','mean'),('VARIANce','var')]
%timeit grouped[['tip_pct','total_bill']].agg(tuples)

4.49 ms ± 630 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
# Another way: tuples
# Can pass aggreation functions as a list of tuples
tuples = [('AvErAGe',np.mean),('VARIANce',np.var)]
%timeit grouped[['tip_pct','total_bill']].agg(tuples)

3.9 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
# Another way: tuples
# Can pass aggreation functions as a list of tuples
tuples = [('AvErAGe','mean'),('VARIANce',np.var)]
%timeit grouped[['tip_pct','total_bill']].agg(tuples)

3.8 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [65]:
# Another way: tuples
# Can pass aggreation functions as a list of tuples
tuples = [('AvErAGe',np.mean),('VARIANce','var')]
%timeit grouped[['tip_pct','total_bill']].agg(tuples)

4.19 ms ± 873 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
# Another way: tuples
# Can pass aggreation functions as a list of tuples
tuples = [('AvErAGe','mean'),('VARIANce','var')]
grouped[['tip_pct','total_bill']].agg(tuples)

tip_pct           total_bill            
              AvErAGe  VARIANce    AvErAGe    VARIANce
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [67]:
# Another way: dictionary
# Applying different functions using a dictionary
grouped.agg({'tip':np.max,
             'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
# Passing aggreation functions as dictionary -> as key-values pair
grouped.agg({'total_bill':['min','max','mean','std'],
             'size':'sum'})

total_bill                              size
                   min    max       mean        std  sum
day  smoker                                             
Fri  No          12.46  22.75  18.420000   5.059282    9
     Yes          5.75  40.17  16.813333   9.086388   31
Sat  No           7.25  48.33  19.661778   8.939181  115
     Yes          3.07  50.81  21.276667  10.069138  104
Sun  No           8.77  48.17  20.506667   8.130189  167
     Yes          7.25  45.35  24.120000  10.442511   49
Thur No           7.51  41.19  17.113111   7.721728  112
     Yes         10.34  43.11  19.190588   8.355149   40

## **Returning Aggregated Data Without Row Indexes**

In [70]:
# Display tips data frame
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [71]:
# Need to pass "numeric_only = True" to mean() as it involves non-numeric data
tips.groupby(['day','smoker']).mean(numeric_only=True)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [73]:
# Disable the index
# Need to pass "numeric_only = True" to mean() as it involves non-numeric data
tips.groupby(['day','smoker'],as_index=False).mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# **10.3 Apply: General split-apply-combine**

In [ ]:
# Define a function called 'top'
# select the top five tip_pct values by group
def top(df, n = 5, column = 'tip_pct'):
    return df.sort_values(column, ascending = False)[:n]

In [ ]:
# Display tips content
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [ ]:
# Apply the top function
# Return top six
# new argument will overwrite the default value in the function
# 'tip_pct' is the default column


,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [ ]:
# groupby 'smoker'
# return top 5 for each 'smoker' group -> No and Yes


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [ ]:
# Pass additional argument after the function
# Overwrite the default values


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
            91        22.49   3.50     No   Fri  Dinner     2  0.155625
            223       15.98   3.00     No   Fri   Lunch     3  0.187735
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
            59        48.27   6.73     No   Sat  Dinner     4  0.139424
            23        39.42   7.58     No   Sat  Dinner     4  0.192288
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
            112       38.07   4.00     No   Sun  Dinner     3  0.105070
            11        35.26   5.00     No   Sun  Dinner     4  0.141804
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
            85        34.83   5.17     No  Thur   Lunch     4  0.148435
            141       34.30   6.70     No  Thur   Lunch     6  0.195335
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
            90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            96        27.28   4.00    Yes   Fri  Dinner     2  0.146628
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
            102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            207       38.73   3.00    Yes   Sat  Dinner     4  0.077459
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
            184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            180       34.65   3.68    Yes   Sun  Dinner     4  0.106205
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982
            83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            192       28.44   2.56    Yes  Thur   Lunch     2  0.090014

In [ ]:
# Using describe() function to get descriptive statistics
# In wide table format


,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [ ]:
# Unstack
# Transform wide table to long format


       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

## **Suppressing the Group Keys**

In [ ]:
# Suppressing group key -> group_keys function


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


## **Quantile and Bucket Analysis**

In [ ]:
# Example dataset
rng = np.random.default_rng(seed=12345)
frame = pd.DataFrame({"data1": rng.standard_normal(1000),
                      "data2": rng.standard_normal(1000)})
frame.head()

,data1,data2
0,-1.423825,-1.176622
1,1.263728,1.205009
2,-0.870662,0.717879
3,-0.259173,-2.686273
4,-0.075343,-1.865260


In [ ]:
# Using the cut() function to transform frame into respective bin
# '( ]' -> left bound is exclusive, right bound is inclusive


0   -1.423825
1    1.263728
2   -0.870662
3   -0.259173
4   -0.075343
Name: data1, dtype: float64

 ------ 



0    (-1.537, 0.0459]
1     (0.0459, 1.629]
2    (-1.537, 0.0459]
3    (-1.537, 0.0459]
4    (-1.537, 0.0459]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.126, -1.537] < (-1.537, 0.0459] < (0.0459, 1.629] <
                                           (1.629, 3.211]]

In [ ]:
# Define a function -> get_stats
def get_stats(group):
    return pd.DataFrame(
        {'min': group.min(), 'max': group.max(),
         'count': group.count(), 'mean': group.mean()}
    )

In [ ]:
# Intermediate object
# quartiles -> four categorical bins


In [ ]:
# Apply the get_stats function


min       max  count      mean
data1                                                      
(-3.126, -1.537] data1 -3.119609 -1.538154     65 -2.019675
                 data2 -2.433466  2.040796     65  0.033381
(-1.537, 0.0459] data1 -1.523449  0.045684    443 -0.605092
                 data2 -2.830704  3.721098    443  0.020211
(0.0459, 1.629]  data1  0.048765  1.621952    438  0.678524
                 data2 -2.320815  3.461976    438  0.029518
(1.629, 3.211]   data1  1.651471  3.211418     54  2.044438
                 data2 -1.778229  2.530599     54  0.228316

In [ ]:
# Using lists of pre-defined standard python functions
# ['min', 'max', 'count', 'mean']


data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.126, -1.537] -3.119609 -1.538154    65 -2.019675 -2.433466  2.040796   
(-1.537, 0.0459] -1.523449  0.045684   443 -0.605092 -2.830704  3.721098   
(0.0459, 1.629]   0.048765  1.621952   438  0.678524 -2.320815  3.461976   
(1.629, 3.211]    1.651471  3.211418    54  2.044438 -1.778229  2.530599   

                                  
                 count      mean  
data1                             
(-3.126, -1.537]    65  0.033381  
(-1.537, 0.0459]   443  0.020211  
(0.0459, 1.629]    438  0.029518  
(1.629, 3.211]      54  0.228316

In [ ]:
# Display top few rows of frame
frame.head()

,data1,data2
0,-1.423825,-1.176622
1,1.263728,1.205009
2,-0.870662,0.717879
3,-0.259173,-2.686273
4,-0.075343,-1.865260


In [ ]:
# Assign each data into respective bin
# Dividing data1 into 4 bins based on their quantiles


0      (-3.121, -0.675]
1        (0.687, 3.211]
2      (-3.121, -0.675]
3      (-0.675, 0.0134]
4      (-0.675, 0.0134]
             ...       
995     (0.0134, 0.687]
996     (0.0134, 0.687]
997    (-0.675, 0.0134]
998     (0.0134, 0.687]
999    (-0.675, 0.0134]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] <
                                           (0.687, 3.211]]

In [ ]:
# Apply the get_stats function
# get_stats -> min, max, count & mean


min       max  count      mean
data1                                                      
(-3.121, -0.675] data1 -3.119609 -0.678494    250 -1.294347
                 data2 -2.830704  3.041653    250  0.062349
(-0.675, 0.0134] data1 -0.673305  0.008009    250 -0.303875
                 data2 -2.686273  3.721098    250 -0.016510
(0.0134, 0.687]  data1  0.018753  0.686183    250  0.343512
                 data2 -2.320804  3.461976    250 -0.025011
(0.687, 3.211]   data1  0.688282  3.211418    250  1.287744
                 data2 -2.320815  3.145451    250  0.124697

## **Filling Missing Values with Group-Specific Values**

In [ ]:
# Example data
rng = np.random.default_rng(12345)
s = pd.Series(rng.standard_normal(6))
s

0   -1.423825
1    1.263728
2   -0.870662
3   -0.259173
4   -0.075343
5   -0.740885
dtype: float64

In [ ]:
# Introduce NaN: skip of two


0         NaN
1    1.263728
2         NaN
3   -0.259173
4         NaN
5   -0.740885
dtype: float64

In [ ]:
# Compute mean of s


0.08789019036970125

In [ ]:
# Fill NA with group mean


0    0.087890
1    1.263728
2    0.087890
3   -0.259173
4    0.087890
5   -0.740885
dtype: float64

In [ ]:
# Another data
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(rng.standard_normal(8), index=states)
data

Ohio         -1.367793
New York      0.648893
Vermont       0.361058
Florida      -1.952863
Oregon        2.347410
Nevada        0.968497
California   -0.759387
Idaho         0.902198
dtype: float64

In [ ]:
# Data wrangling / transformation
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.367793
New York      0.648893
Vermont            NaN
Florida      -1.952863
Oregon        2.347410
Nevada             NaN
California   -0.759387
Idaho              NaN
dtype: float64

In [ ]:
# Size based on the 'group_key'


East    4
West    4
dtype: int64

In [ ]:
# Display data content
data

Ohio         -1.367793
New York      0.648893
Vermont            NaN
Florida      -1.952863
Oregon        2.347410
Nevada             NaN
California   -0.759387
Idaho              NaN
dtype: float64

In [ ]:
# count based on the 'group_key'
# Nan is not counted


East    3
West    2
dtype: int64

In [ ]:
# Define a custom function -> fill_mean
def fill_mean(group):
    return group.fillna(group.mean())

In [ ]:
# Display content of data
data

Ohio         -1.367793
New York      0.648893
Vermont            NaN
Florida      -1.952863
Oregon        2.347410
Nevada             NaN
California   -0.759387
Idaho              NaN
dtype: float64

In [ ]:
# Fill the NA values using the group means


East  Ohio         -1.367793
      New York      0.648893
      Vermont      -0.890588
      Florida      -1.952863
West  Oregon        2.347410
      Nevada        0.794011
      California   -0.759387
      Idaho         0.794011
dtype: float64

In [ ]:
# Manual calculation
(-1.367793 + 0.648893 - 1.952863) / 3

-0.8905876666666667

# **Group Weighted Average and Correlation**

In [ ]:
# An example data
rng = np.random.RandomState(12345)
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": rng.standard_normal(8),
                   "weights": rng.uniform(size=8)})
df

,category,data,weights
0,a,-0.204708,0.748907
1,a,0.478943,0.653570
2,a,-0.519439,0.747715
3,a,-0.555730,0.961307
4,b,1.965781,0.008388
5,b,1.393406,0.106444
6,b,0.092908,0.298704
7,b,0.281746,0.656411


In [ ]:
# Intermediate object groupby category
grouped = df.groupby('category')

# Define a weighted average function -> get_wavg
def get_wavg(group):
    return np.average(group['data'], weights=group['weights'])

In [ ]:
# Apply get_wavg function


category
a   -0.245188
b    0.352824
dtype: float64

# **Weighted Mean Formula**:

* find the ***sum of all the variables multiplied by their weight***, then ***divide by the sum of the weights***.

* For more information about Weighted mean, visit this website: http://tinyurl.com/4d4nr7fh

In [ ]:
# Display content of df
df

,category,data,weights
0,a,-0.204708,0.748907
1,a,0.478943,0.653570
2,a,-0.519439,0.747715
3,a,-0.555730,0.961307
4,b,1.965781,0.008388
5,b,1.393406,0.106444
6,b,0.092908,0.298704
7,b,0.281746,0.656411


In [ ]:
# Calculate weighted mean manually
(-0.204708*0.748907 + 0.478943*0.653570 + (-0.519439* 0.747715) + (-0.555730*0.961307)) / (0.748907+ 0.653570 + 0.747715 + 0.961307)

-0.24518855658992655

In [ ]:
# Another example
# download the stock_px.csv here: https://bit.ly/3vuuejJ
# end-of-day prices for a Apple, Microsoft, ExxonMobil and the S&P 500 index
# parse_dates=True -> parse any columns that look like dates into datetime objects
# index_col=0 -> specifies which column from the CSV file should be used as the index
close_px = pd.read_csv('https://bit.ly/3vuuejJ',
                       parse_dates=True,
                       index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [ ]:
# Check top few rows of close_px
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [ ]:
# Define a custom spx_corr function
# pair-wise correlation of each column with the "SPX" column
def spx_corr(group):
    return group.corrwith(group['SPX'])

In [ ]:
# pct_change() -> Calculate percentage change between the current and prior value


,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [ ]:
# Define a custom get_year function
def get_year(x):
    return x.year

In [ ]:
# Group by year
# by_year is an intermediate object


In [ ]:
# Apply spx_corr function


,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [ ]:
# Define a custom corr_aapl_msft function
# compute inter-column correlations
def corr_aapl_msft(group):
    return group['AAPL'].corr(group['MSFT'])

In [ ]:
# Annual correlation between Apple and Microsoft


2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# **Group-Wise Linear Regression**

In [ ]:
# Importing the statsmodels library
# statsmodels -> Python library to do statistical analysis
import statsmodels.api as sm

In [ ]:
# Define a custom ordinary least squares (OLS) regression function
# to find the best-fitting line by minimizing the sum of squared differences between observed and predicted values.
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
# Yearly linear regression of AAPL on SPX returns
# AAPL is part of Magnificent 7 in SPX500
# Apple (AAPL), Alphabet (GOOGL, GOOG), Microsoft (MSFT), Amazon (AMZN), Meta (META), Nvidia (NVDA), & Tesla (TSLA)
# AAPL = .00071 + 1.195406SPX
# Refer to this link for AAPL stock price between 2007 and 2009: http://tinyurl.com/4cs9ppp2


,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# **10.4 Group Transforms and “Unwrapped” GroupBys**

In [ ]:
# An example
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [ ]:
# group means by key
g = df.groupby('key', group_keys=True)['value'] # create intermediate object
g.mean() # compute mean

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [ ]:
# Define a custom normalization function
def normalise(x):
    return (x - x.mean()) / x.std()

In [ ]:
# method 1 - slower (user-defined function, preseumably not optimised)
%timeit g.apply(normalise)

3.25 ms ± 87.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# method 2 - fast (user-defined function, preseumably not optimised)
%timeit g.transform(normalise)

3.23 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# Faster
# unwrapped group operations - doing arithmetic between the outputs of
# multiple GroupBy operations
# often much faster than a general apply function
%timeit (df['value'] - g.transform('mean')) / g.transform('std')

1.17 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# ***Traditional group operation***

* involve aggregating the data within each group using functions like mean, sum, etc.
* These functions reduce the data within each group to a single value (the aggregate)

# ***Unwrapped group operation***

* refer to the ability to perform operations on grouped data ***without explicitly aggregating the results***
* apply a function to each group in a DataFrame, and the function can return a ***modified version*** of the data ***rather than a single aggregated value***



# ***Still built-in aggregation functions are the fastest***
* they are ***implemented in C and optimized for performance***

In [ ]:
# Fastest - using default python function
# often highly optimized and can be very fast
# built-in aggregation functions are likely to be faster because they are implemented in C and optimized for performance
%timeit g.transform('mean')

403 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# **Transform vs Apply**

* The ***transform*** method is generally ***more efficient than apply*** when we are performing a simple element-wise operation on each group
* The ***apply method*** is ***more flexible but can be slower than transform*** for certain operations.
    * apply method allows more complex operations, and the function passed to apply can return a scalar, a Series, or a DataFrame.
* if the operation is simple, ***transform is often faster***.

# **10.5 Pivot Tables and Cross-Tabulation**
A ***pivot table*** is a ***data summarization*** tool

In [ ]:
# From previous tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [ ]:
# pivot table - getting mean for each column
# A groupby function


<ipython-input-140-d553d1836c7c>:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  tips.pivot_table(index=['day', 'smoker'])


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [ ]:
# Another example - getting mean for size and tip_pct columns
# Complex but very useful


size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

## **Cross-Tabulations: Crosstab**
A ***special pivot table*** to compute ***group frequencies***

In [ ]:
# Import a specific class or object called StringIO from the io module
# allows us to read from or write to strings as if they were files
from io import StringIO

In [ ]:
# data consists of strings
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

In [ ]:
# Create a table
# sep="\s+" -> indicates that one or more whitespace characters are used as the separator between columns
# We called this as raw dataframe
data = pd.read_table(StringIO(data), sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [ ]:
# Crosstab example 1 - find group frequencies
# summarize by nationality and handedness
# Secondary information by doing some summarization
# margins=True -> to ensure sum of rows and columns are the same
# provide a quick overview of the overall distribution of data across the different combinations of data


Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [ ]:
# tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


# ***Crosstab always have two comparisons***:
* list A vs List B
* [tips['time'], tips['day']] as List A
* tips['smoker'] as List B

In [ ]:
# Crosstab example 2


smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

# **That's all for the day!!!**

See you all next week :)